#### Adam Klie<br>05/07/2020
# Perform core microbiome analyses in Qiime2
1. Taxonomic classification
2. Rarefaction plots
3. Core diversity
4. Diffential abundance
5. Compositionally aware diversity
6. Sample classification

In [200]:
import os
import qiime2
import warnings
warnings.filterwarnings('ignore')
from qiime2.plugins import feature_table, taxa, diversity, songbird, metadata, deicode, emperor, qurro, sample_classifier

## Directory set-up and data loading

In [2]:
# Data and results directories
data_dir = '../data/test/2020_05_05'
results_dir = '../results/test/2020_05_06'

In [155]:
# Specific results directories
tax_dir = '{}/taxonomy'.format(results_dir)
table_dir = '{}/feature_tables'.format(results_dir)
tree_dir = '{}/fragment_insertion'.format(results_dir)
diversity_dir = '{}/diversity'.format(results_dir)
diff_dir = '{}/differential_abundance'.format(results_dir)

In [170]:
# Load feature table and meta_data
f_table = qiime2.Artifact.load('{}/insertion-table.qza'.format(table_dir))
meta_data = qiime2.Metadata.load('{}/metadata.tsv'.format(data_dir)) 

In [5]:
# Filter meta_data for faster plotting
filtered_meta_data = qiime2.Metadata(meta_data.to_dataframe()[["age_corrected", "sex", "body_site", 
                                                               "sample_type", "bmi_corrected", "exercise_frequency", 
                                                               "country", "race",  "diet_type"]])

## Taxonomy analysis
 - Inputs: 
     - insertion-table.qza (FeatureData[Frequency] artifact)
     - bespoke-taxonomy.qza (FeatureData[Taxonomy] artifact)
     - metadata.tsv (tab seperated table)
 - Outputs: 
     - taxa-bar-plot.qzv (Visualization)
 - Plugins: 
     - taxa

In [36]:
# Load taxonomy artifact
taxonomic_class = qiime2.Artifact.load('{}/bespoke-taxonomy.qza'.format(tax_dir))

In [ ]:
# Generate barplot of taxa
taxonomic_class_vis = taxa.visualizers.barplot(table=feature_table,
                                               taxonomy=taxonomic_class,
                                               metadata=meta_data)
#taxonomic_class_vis.visualization

In [ ]:
# Save the barplot as visualization
taxonomic_class_vis.visualization.save('{}/taxa-bar-plot.qzv'.format(tax_dir))

## Rarefaction plots
 - Inputs: 
     - insertion-table.qza (FeatureData[Frequency] artifact)
     - sepp-tree.qza (Phylogeny[Rooted] artifact)
     - metadata.tsv (tab seperated table) -- highly recommend using filtered metadata here
 - Outputs: 
     - alpha-rarefaction.qzv (Visualization)
 - Plugins: 
     - diversity

In [6]:
# Load the insertion tree
tree = qiime2.Artifact.load('{}/sepp-tree.qza'.format(tree_dir))

In [33]:
# Create rarefaction plots, takes about 4 minutes with ~30 samples and 9 metadata columns
alpha_rarefaction_vis = diversity.visualizers.alpha_rarefaction(table=feature_table,
                                                                max_depth=10000,
                                                                phylogeny=tree,
                                                                metadata=filtered_meta_data)
#alpha_rarefaction_vis.visualization

CPU times: user 4min 1s, sys: 9.87 s, total: 4min 11s
Wall time: 4min 19s


In [41]:
# Save the visualization
alpha_rarefaction_vis.visualization.save('{}/alpha-rarefaction.qzv'.format(diversity_dir))

'../results/test/2020_05_06/diversity/alpha-rarefaction.qzv'

## Core diversity
 - Inputs: 
     - insertion-table.qza (FeatureData[Frequency] artifact)
     - sepp-tree.qza (Phylogeny[Rooted] artifact)
     - metadata.tsv (tab seperated table) -- highly recommend using filtered metadata here
 - Outputs: 
     - rarefied-table.qza (FeatureTable[Frequency]
     - rarefied-table.qzv (Visualization)
     - shannon-vector.qza SampleData[AlphaDiversity]
     - faith-pd-vector.qza SampleData[AlphaDiversity]
     - faith-pd-group-significance.qzv (Visualization)
     - uw-unifrac-exercise-significance.qzv (Visualization)
     - uw-unifrac-emperor.qzv (Visualization)
 - Plugins: 
     - diversity
     - feature-table

In [137]:
# Depth to rarefy to, will lose all samples below this depth and sample from those above
depth = 5000

In [12]:
# Perform core diversity analysis
core_diversity = diversity.pipelines.core_metrics_phylogenetic(feature_table,
                                                               phylogeny=tree,
                                                               sampling_depth=depth,
                                                               metadata=filtered_meta_data,
                                                               n_jobs=1)

In [141]:
# Visualize the rarefied feature table used for analysis
rarefied_table_vis = feature_table.visualizers.summarize(table=core_diversity.rarefied_table, sample_metadata=filtered_meta_data)

In [147]:
# Save vectors and rarefied table
core_diversity.rarefied_table.save('{}/rarefied-table.qza'.format(table_dir))
rarefied_table_vis.visualization.save('{}/rarefied-table.qzv'.format(table_dir))
core_diversity.shannon_vector.save('{}/shannon-vector.qza'.format(diversity_dir))
core_diversity.faith_pd_vector.save('{}/faith-pd-vector.qza'.format(diversity_dir))
core_diversity.unweighted_unifrac_distance_matrix.save('{}/uw-unifrac-distance-matrix.qza'.format(diversity_dir))

'../results/test/2020_05_06/diversity/uw-unifrac-distance-matrix.qza'

### A. Alpha diversity significance testing
As recommended by Bod, test alpha significance for shannon diversity and faith's pd
 - Input
     - shannon-vector.qza (SampleData[AlphaDiversity])
     - faith-pd-vector.qza (SampleData[AlphaDiversity])
 - Output
     - shannon-group-significance.qzv (Visualization)
     - faith-pd-group-significance.qzv (Visualization)
 - Plugin
     - diversity

In [45]:
# Test and visualize shannon diversity for group significance
shannon_div = diversity.visualizers.alpha_group_significance(alpha_diversity=core_diversity.shannon_vector, metadata=filtered_meta_data)
#shannon_div.visualization

In [47]:
# Test and visualize faith's pd diversity for group significance
faiths_div = diversity.visualizers.alpha_group_significance(alpha_diversity=core_diversity.faith_pd_vector, metadata= filtered_meta_data)
#faiths_div.visualization

In [57]:
# Save significance visualizations
shannon_div.visualization.save('{}/shannon-group-significance.qzv'.format(diversity_dir))
faiths_div.visualization.save('{}/faith-pd-group-significance.qzv'.format(diversity_dir))

'../results/test/2020_05_06/diversity/faith-pd-group-significance.qzv'

### B. Beta diversity significance testing
As recommended by Bod, test beta diversity significance using unweighted unifrac and visualize emperor plor
 - Input
     - uw-unifrac-distance-matrix.qza (DistanceMatrix % Properties('phylogenetic'))
     - metadata.tsv (tab seperated table)
 - Output
     - uw-unifrac-exercise-significance/qzv (Visualization)
     - uw-unifrac-emperor.qzv (Visualization)
 - Plugin
     - diversity

In [197]:
# Remove '/' in exercise column values (https://forum.qiime2.org/t/plugin-error-from-diversity-alpha-rarefaction/4548/4)
tmp_df = filtered_meta_data.to_dataframe()
tmp_df['exercise_frequency'] = tmp_df['exercise_frequency'].map(lambda x: str(x).replace('/', ' a '))
filtered_meta_data = qiime2.Metadata(tmp_df)

In [81]:
# Test and visualize unweighted unifrac for group significance
uw_unifrac_div = diversity.visualizers.beta_group_significance(core_diversity.unweighted_unifrac_distance_matrix,
                                                               metadata=filtered_meta_data.get_column('exercise_frequency'),
                                                               pairwise=True)
#uw_unifrac_div.visualization

In [148]:
# Visualize emperoro plot of unweighted unifrac
#core_diversity.unweighted_unifrac_emperor

In [86]:
# Save the group significance and emperor visualizations
uw_unifrac_div.visualization.save('{}/uw-unifrac-exercise-significance.qzv'.format(diversity_dir))
core_diversity.unweighted_unifrac_emperor.save('{}/uw-unifrac-emperor.qzv'.format(diversity_dir))

'../results/test/2020_05_06/diversity/uw-unifrac-emperor.qzv'

### C. Beta diversity significance testing with confounders using ADONIS
As recommended by Bod, test beta diversity significance using ADONIS for confounders
 - Input
     - uw-unifrac-distance-matrix.qza
 - Output
     - adonis-exercise-significance.qzv
 - Plugin
     - diversity
 - TODO
     - Decide a formula
 - Help

In [218]:
# Perform covariate aware PERMANOVA
adonis_vis = diversity.visualizers.adonis(distance_matrix=core_diversity.unweighted_unifrac_distance_matrix,
                            metadata=filtered_metadata,
                            formula="exercise_frequency+sex")
#adonis_vis.visualization

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /var/folders/tm/f8q3_wcj5s98dwjv86bvq7hr0000gn/T/tmprfsx1ow1/dm.tsv /var/folders/tm/f8q3_wcj5s98dwjv86bvq7hr0000gn/T/tmprfsx1ow1/md.tsv exercise_frequency+sex 999 1 /var/folders/tm/f8q3_wcj5s98dwjv86bvq7hr0000gn/T/qiime2-temp-glmc1rsf/adonis.tsv



In [219]:
# Save the visualization
adonis_vis.visualization.save('{}/adonis-exercise-significance.qzv'.format(diversity_dir))

'../results/test/2020_05_06/diversity/adonis-exercise-significance.qzv'

### D. Compositionally aware beta diversity using DEICODE 
 - Input
     - insertion-table.qza
 - Output
     - deicode-biplot.qza
     - deicode-distance-matrix.qza
     - deicode-emperor.qzv
     - deicode-exercise-significance.qzv
 - Plugins
     - deicode
     - diversity
 - TODO
     - 
 - Help
     - https://library.qiime2.org/plugins/deicode/19/
     - https://github.com/biocore/DEICODE
     - https://forum.qiime2.org/t/robust-aitchison-pca-beta-diversity-with-deicode/8333
     - https://nbviewer.jupyter.org/github/biocore/DEICODE/blob/master/ipynb/tutorials/moving-pictures.ipynb
     - https://nbviewer.jupyter.org/github/biocore/DEICODE/blob/master/ipynb/tutorials/moving-pictures-standalone-cli-and-api.ipynb

In [208]:
# Make a deicode subdirectory
deicode_dir = '{}/deicode'.format(diversity_dir)
if not os.path.exists(deicode_dir): 
    os.makedirs(deicode_dir)

In [186]:
# Run robust Atchison PCA
deicode_result = deicode.methods.auto_rpca(table=f_table)

In [190]:
# Visualize the PCoA biplot generated
feature_meta_data = taxonomic_class.view(qiime2.Metadata)
deicode_vis = emperor.visualizers.biplot(biplot=deicode_result.biplot, 
                                         sample_metadata=filtered_meta_data, 
                                         feature_metadata=feature_meta_data)
#deicode_vis.visualization

In [198]:
# Perform PERMANOVA on exercise frequency
deicode_sig = diversity.visualizers.beta_group_significance(distance_matrix=deicode_result.distance_matrix,
                                                           metadata=filtered_meta_data.get_column('exercise_frequency'),
                                                           method='permanova')
#deicode_sig.visualization

In [213]:
# Save results and visualization
deicode_result.biplot.save('{}/deicode-biplot.qza'.format(deicode_dir))
deicode_result.distance_matrix.save('{}/deicode-distance-matrix.qza'.format(deicode_dir))
deciode_vis.visualization.save('{}/deicode-emperor.qzv'.format(deicode_dir))
deicode_sig.visualization.save('{}/deicode-exercise-significance.qzv'.format(deicode_dir))

'../results/test/2020_05_06/diversity/deicode/deicode-exercise-significance.qzv'

## Differential abundance with Songbird
 - Input
     - insertion-table.qza (FeatureTable[Frequency])
     - metadata.tsv
     - formula (String)
 - Output
     - regression-stats.qza (SampleData[SongbirdStats])
     - regression-stats.qzv (Visualization)
     - differentials.qza (FeatureData[Differential])
     - differentials.qzv (Visualization)
 - TODO
     - When running full dataset, will need fine tune error curves
     - Will need to decide exactly what formula I want (necessary covariates for baseline, add exercise for test
     - Question we are trying to answer is does adding exercise_frequency contribute to taxa abundance prediction
     - Fully understand differential abundances table
 - Help
     - https://github.com/biocore/songbird/
     - https://github.com/biocore/songbird/issues/126
     - https://github.com/knightlab-analyses/reference-frames

In [105]:
# Model with exercise_frequency as covariate
songbird_result = songbird.methods.multinomial(table=feature_table,
                                              metadata=filtered_meta_data,
                                              formula="exercise_frequency",
                                              epochs=1000,
                                              differential_prior=0.1,
                                              summary_interval=1)

100%|██████████| 5000/5000 [00:08<00:00, 556.75it/s]


In [106]:
# Null model
null_result = songbird.methods.multinomial(table=feature_table,
                                              metadata=filtered_meta_data,
                                              formula="1",
                                              epochs=1000,
                                              differential_prior=0.1,
                                              summary_interval=1)

100%|██████████| 5000/5000 [00:08<00:00, 609.95it/s]


In [152]:
# Visualize error curves for baseline vs model
songbird_vis = songbird.visualizers.summarize_paired(regression_stats=songbird_result.regression_stats, baseline_stats=null_result.regression_stats)
#songbird_vis.visualization

In [154]:
# Visualize the differential abundances via a table
differentials = metadata.visualizers.tabulate(songbird_result.differentials.view(qiime2.Metadata))
#differentials.visualization

In [161]:
# Save error plots and differentials
songbird_result.regression_stats.save('{}/regression-stats'.format(diff_dir))
songbird_vis.visualization.save('{}/regression-stats'.format(diff_dir))
songbird_result.differentials.save('{}/differentials'.format(diff_dir))
differentials.visualization.save('{}/differentials'.format(diff_dir))

'../results/test/2020_05_06/differential_abundance/differentials.qzv'

## Qurro
 - Input
     - deicode
     - songbird
 - Output
     -
 - Plugin
     - qurro
 - TODO
     -
 - Help
     - https://github.com/biocore/qurro
     - https://nbviewer.jupyter.org/github/biocore/qurro/blob/master/example_notebooks/moving_pictures/moving_pictures.ipynb

In [163]:
#taxonomic_class.view(qiime2.Metadata).to_dataframe()

In [134]:
qurro.visualizers.loading_plot(deicode_result.biplot,
                              table=feature_table,
                              sample_metadata=filtered_meta_data,
                              feature_metadata=try_did).visualization

2543 feature(s) in the BIOM table were not present in the feature rankings.
These feature(s) have been removed from the visualization.
65 sample(s) in the sample metadata file were not present in the BIOM table.
These sample(s) have been removed from the visualization.


<visualization: Visualization uuid: 9311432f-b27d-4afb-9f74-a460f0a76be9>

## Sample classification
 - Input
 - Output
 - Plugins
     - sample-classifier
 - TODO
 - Help